In [1]:
import re
import random
from tqdm.notebook import tqdm
from tika import parser

import warnings
from pathlib import Path

import spacy
from thinc.api import SGD
from spacy.tokens import DocBin

/home/admin/gopi/envs/dlarith/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_file = '/Users/gopinath.balu/Downloads/text_extraction/data/Use caes NLP.pdf'

In [3]:
raw = parser.from_file(pdf_file)
content = raw['content']

In [4]:
remove_newline = lambda x: x.replace('\n', ' ').strip()

In [ ]:
actual_content = remove_newline(content)

In [2]:
actual_content = "POS. DESCRIPTION QUANTITE   COMMANDEE QUANTITE   CONFIRMEE PRIX /  UNITE  NET /  UNITE  MONTANT  NET  10  PFWVTG001005000803 / PWVTGG50080 Z (*) Vis à panneaux PFS+ TF-T * 5,00x80 Zn (T25)  double tête plate six lobes int. avec nervures  filet partiel - pointe coupante T17  emballage standard Art. client: 924734 QTE: 200 Pce  Demandé: 2.000 Pce Arrondi:  2.400 Pce  Stock: 2.400 Pce  49,97 / 1.000   Pce  -2 %  48,97 / 1.000   Pce  117,53   20  PFWVTV001004500603 / PWVTVK45060 Z (*) Vis à panneaux PFS+ TF-T * 4,50x60 Zn (T20)  double tête plate six lobes int. avec nervures  filet total - pointe coupante T17  emballage petit Art. client: 929735 QTE: 200 Pce  2.400 Pce Stock: 2.400 Pce  28,63 / 1.000   Pce   28,63 / 1.000   Pce  68,71   Sous Total 188,64   Remise cumulée -2,40   Montant net 186,24   TVA (21%) 39,11   Montant total 225,35  Poids brut 27,600 KG"

In [3]:
edited_content = "POS. DESCRIPTION QUANTITE   COMMANDEE QUANTITE   CONFIRMEE PRIX /  UNITE  NET /  UNITE  MONTANT  NET  10  PFWVTG001005000803 / PWVTGG50080 Z (*) Vis à panneaux PFS+ TF-T * 5,00x80 Zn (T25)  double tête plate six lobes int. avec nervures  filet partiel - pointe coupante T17  emballage standard Art. client: {} QTE: 200 Pce  Demandé: 2.000 Pce Arrondi:  2.400 Pce  Stock: 2.400 Pce  {}   Pce  -2 %  {}   Pce  117,53   20  PFWVTV001004500603 / PWVTVK45060 Z (*) Vis à panneaux PFS+ TF-T * 4,50x60 Zn (T20)  double tête plate six lobes int. avec nervures  filet total - pointe coupante T17  emballage petit Art. client: {} QTE: 200 Pce  2.400 Pce Stock: 2.400 Pce  {}   Pce   {}   Pce  68,71   Sous Total 188,64   Remise cumulée -2,40   Montant net 186,24   TVA (21%) 39,11   Montant total 225,35  Poids brut 27,600 KG"

In [4]:
#items_to_extract = ['Art. client: 924734', '49,97 €  / 1.000', '48,97 €  / 1.000', 'Art. client: 929735', '28,63 €  / 1.000', '28,63 €  / 1.000']

In [4]:
art_clients = [f'{random.randint(10000, 999999)}' for i in range(100)]
prices1 = [f'{random.randint(1, 999)},{random.randint(0, 99)} / 1.000' for i in range(100)]
prices2 = [f'{random.randint(1, 999)},{random.randint(0, 99)} / 1.000' for i in range(100)]

In [5]:
global info_corpus
info_corpus = list(zip(art_clients, prices1, prices2))
# info_corpus = info_corpus[:10]

In [6]:
def shuffle_word(input_sentence: str) -> str:
    """Shuffle the input string

    Args:
        word (str): Input string

    Returns:
        str: Shuffled output
    """

    words = input_sentence.split(' ')
    random.shuffle(words)
    return ' '.join(words)

def get_items(n: int) -> list:
    """Get generated info from the list

    Args:
        n (int): N no of items to fetch

    Returns:
        list: return list
    """

    if len(info_corpus) > n:
        result = random.sample(info_corpus, n)
        for item in result:
            info_corpus.remove(item)
    else:
        result = random.sample(info_corpus, len(info_corpus))
        for item in result:
            info_corpus.remove(item)
    return result

def insert_entries(doc_content: str, entities_list: list) -> str:
    """Insert given entities into the document content

    Args:
        doc_content (str): Document content
        entities_list (list): List of entities

    Returns:
        str: Document with the inserted entities
    """
    entity_label_list = list()
    return_entity_dict = dict()
    # doc_length = len(doc_content.split())
    # randidx = random.randint(50, int(doc_length/2))
    
    # for artid, price1, price2 in entities_list:
    #     doc_word_list = doc_content.split(' ')
    #     # doc_word_list.insert(randidx, artid)
    #     doc_word_list.insert(randidx+random.choice([32, 48]), price1)
    #     doc_word_list.insert(randidx+random.choice([64, 128]), price2)
    #     doc_content = ' '.join([str(i) for i in doc_word_list])
    #     randidx = randidx + random.choice([32, 64])
    
    merged_entities_list = entities_list[0] + entities_list[1]
    doc_content = doc_content.format(*merged_entities_list)
    
    for items in entities_list:
        for item in items:
            if '/' not in item.lower():
                start, end = re.search(item, doc_content).span()
                entity_label_list.append((start, end, 'Art ID'))
                pass
            else:
                start, end = re.search(item, doc_content).span()
                entity_label_list.append((start, end, 'Price per Unit'))
    return_entity_dict['entities'] = entity_label_list
        
        
    return [doc_content, return_entity_dict]

In [7]:
def generate_synthetic_data(doc_content: str, data_corpus: list) -> None:
    """To generate synthetic data to train SpaCy custom NER model. 
    This returns None but generates a training data.

    Args:
        doc_content (str): Input document content, which will be 
                           modified.
        data_corpus (list): Input list of items to be placed 
                            randomly in the document.
    """
    


    data_list = list()
    len_doc = len(doc_content)

    while len(info_corpus) > 0:
        no_of_samples = 2 #random.randint(1, 2)
        # new_doc_content = shuffle_word(doc_content)
        new_doc_content = doc_content
        entities = get_items(no_of_samples)
        data_list.append(insert_entries(new_doc_content, entities))
        
    return data_list

TRAINING_DATA = generate_synthetic_data(edited_content, info_corpus)


In [9]:
def convert(lang: str, TRAINING_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAINING_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [10]:
convert("fr", TRAINING_DATA, "/home/admin/gopi/Workspace/text_extraction/data/fr_model2/train.spacy")


In [11]:
model = 'fr_core_news_md'
optimizer = SGD(learn_rate=0.001, L2=1e-6, grad_clip=1.0)
losses = {}

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('fr')  
    print("Created blank 'fr' model")

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
    ner.add_label("O")
else:
    ner = nlp.get_pipe('ner')

Loaded model 'fr_core_news_md'


In [12]:
from spacy.training.example import Example

for batch in spacy.util.minibatch(TRAINING_DATA, size=2):
    for text, annotations in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        # nlp.update([example], sgd=optimizer, losses=losses, drop=0.3)
        nlp.update([example], losses=losses, drop=0.3)
        print("Losses", losses)

nlp.to_disk('/home/admin/gopi/Workspace/text_extraction/output/fr_model2')

Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 49.524407568274036}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 92.9774876906314}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 129.8575642896357}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 152.87631826737461}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 172.99090532587257}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 189.06375478374707}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 202.5538351176792}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 215.44947408671666}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 226.91382889794565}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 238.6083827823296}
Losses {'tok2vec': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'ner': 249.6635642103138}
Losses {'tok2vec': 0.0, 'morphologizer

In [13]:
doc = nlp(actual_content)
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('924734', 'Art ID'), ('49,97 / 1.000', 'Price per Unit'), ('48,97 / 1.000', 'Price per Unit'), ('929735', 'Art ID'), ('28,63 / 1.000', 'Price per Unit'), ('28,63', 'Art ID')]
